In [5]:
# regular imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import env
from env import host, user, password

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

# Wrangling
from sklearn.model_selection import train_test_split

In [6]:

def get_db_url(db):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
sql_query = '''
SELECT prop. *,
predictions_2017.logerror,
predictions_2017.transactiondate,
air.airconditioningdesc,
arch.architecturalstyledesc,
build.buildingclassdesc,
heat.heatingorsystemdesc,
land.propertylandusedesc,
story.storydesc,
type.typeconstructiondesc
FROM properties_2017 prop
JOIN (
            SELECT parcelid, MAX(transactiondate) AS max_transactiondate
            FROM predictions_2017
            GROUP BY parcelid) pred USING(parcelid)
JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
AND pred.max_transactiondate = predictions_2017.transactiondate
LEFT JOIN airconditioningtype air USING(airconditioningtypeid)
LEFT JOIN architecturalstyletype arch USING(architecturalstyletypeid)
LEFT JOIN buildingclasstype build USING(buildingclasstypeid)
LEFT JOIN heatingorsystemtype heat USING(heatingorsystemtypeid)
LEFT JOIN propertylandusetype land USING(propertylandusetypeid)
LEFT JOIN storytype story USING(storytypeid)
LEFT JOIN typeconstructiontype type USING(typeconstructiontypeid)
WHERE propertylandusedesc IN ('Single Family Residential' , 'Mobile Home', 'Manufactured, Modular, Prefabricated Homes', 'Patio Home', 'Bungalow', 'Planned Unit Development') 
AND transactiondate <= '2017-12-31'
AND prop.longitude IS NOT NULL
AND prop.latitude IS NOT NULL;
'''
    
def get_zillow_data():
    df = pd.read_sql(sql_query, get_db_url('zillow'))
    df = df.drop(columns='id')
    return df

In [10]:
df = get_zillow_data()

In [11]:
#Summary statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,"54,395.00","12,968,798.63","3,495,113.43","10,711,855.00","11,481,340.50","12,540,352.00","14,105,468.00","167,689,317.00"
airconditioningtypeid,"15,320.00",2.28,3.66,1.00,1.00,1.00,1.00,13.00
architecturalstyletypeid,70.00,7.10,2.67,2.00,7.00,7.00,7.00,21.00
basementsqft,47.00,678.98,711.83,38.00,263.50,512.00,809.50,"3,560.00"
bathroomcnt,"54,395.00",2.32,1.02,0.00,2.00,2.00,3.00,18.00
bedroomcnt,"54,395.00",3.30,0.95,0.00,3.00,3.00,4.00,14.00
buildingqualitytypeid,"35,647.00",6.35,1.72,1.00,5.00,6.00,8.00,12.00
calculatedbathnbr,"54,221.00",2.33,1.02,1.00,2.00,2.00,3.00,18.00
decktypeid,392.00,66.00,0.00,66.00,66.00,66.00,66.00,66.00
finishedfloor1squarefeet,"4,381.00","1,546.23",674.94,184.00,"1,151.00","1,410.00","1,760.00","6,912.00"


In [12]:
#Info on the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54395 entries, 0 to 54394
Data columns (total 67 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      54395 non-null  int64  
 1   airconditioningtypeid         15320 non-null  float64
 2   architecturalstyletypeid      70 non-null     float64
 3   basementsqft                  47 non-null     float64
 4   bathroomcnt                   54395 non-null  float64
 5   bedroomcnt                    54395 non-null  float64
 6   buildingclasstypeid           0 non-null      object 
 7   buildingqualitytypeid         35647 non-null  float64
 8   calculatedbathnbr             54221 non-null  float64
 9   decktypeid                    392 non-null    float64
 10  finishedfloor1squarefeet      4381 non-null   float64
 11  calculatedfinishedsquarefeet  54291 non-null  float64
 12  finishedsquarefeet12          54085 non-null  float64
 13  f

In [13]:
print(f'r0ws: {df.shape[0]}')
print(f'co1umns: {df.shape[1]}')

r0ws: 54395
co1umns: 67


In [14]:
for col in df.columns:
    print(col)
    print(df[col].value_counts())

parcelid
14297519    1
12050704    1
11217239    1
13967614    1
12865580    1
           ..
12410258    1
10970834    1
14171698    1
12893364    1
12826780    1
Name: parcelid, Length: 54395, dtype: int64
airconditioningtypeid
 1.00    13578
13.00     1567
 5.00      159
11.00       16
Name: airconditioningtypeid, dtype: int64
architecturalstyletypeid
 7.00    62
 3.00     3
 2.00     2
21.00     2
 8.00     1
Name: architecturalstyletypeid, dtype: int64
basementsqft
  900.00    2
  640.00    2
  100.00    2
  515.00    2
  273.00    2
  912.00    2
  314.00    1
  819.00    1
1,809.00    1
  604.00    1
  126.00    1
  588.00    1
  786.00    1
  669.00    1
  800.00    1
  396.00    1
  112.00    1
  645.00    1
  384.00    1
1,969.00    1
  252.00    1
  600.00    1
1,218.00    1
  280.00    1
  300.00    1
  224.00    1
  512.00    1
  380.00    1
  204.00    1
  200.00    1
   90.00    1
3,112.00    1
  405.00    1
1,252.00    1
  254.00    1
3,560.00    1
  352.00    1
  168.00

Nulls in data

In [15]:
df.isnull().head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,False,True,True,True,False,False,True,True,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,True,False,False,False,True,True,True,True,True,False,False,True,False,False,False,True,False,False,True,False,True,True,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
1,False,True,True,True,False,False,True,True,False,True,False,False,False,True,True,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,True,False,False,True,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
2,False,True,True,True,False,False,True,True,False,True,True,False,False,True,True,True,True,False,True,False,False,False,True,True,False,False,False,False,True,True,True,False,False,False,True,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,True,True,False,False,False,True,True,True,True,False,True,True
3,False,True,True,True,False,False,True,False,False,True,True,False,False,True,True,True,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,True,False,False,True,True,True,False,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,True,True,True,False,False,True,True
4,False,False,True,True,False,False,True,False,False,True,True,False,False,True,True,True,True,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,True,True,True,False,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,True


In [16]:
df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,NaN,NaN,NaN,3.50,4.00,None,NaN,3.50,NaN,NaN,"3,100.00","3,100.00",NaN,None,NaN,NaN,"6,059.00",NaN,3.00,2.00,633.00,NaN,NaN,"33,634,931.00","-117,869,207.00","4,506.00",NaN,NaN,NaN,NaN,NaN,122,261.00,None,"60,590,630.07","53,571.00","1,286.00",NaN,"96,978.00",0.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,998.00",NaN,NaN,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",None,NaN,"60,590,630,072,012.00",0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,NaN,NaN,NaN,1.00,2.00,None,NaN,1.00,NaN,"1,465.00","1,465.00","1,465.00",NaN,None,"1,465.00",NaN,"6,111.00",1.00,1.00,1.00,0.00,NaN,NaN,"34,449,266.00","-119,281,531.00","12,647.00",NaN,NaN,NaN,NaN,NaN,1110,261.00,None,"61,110,010.02","13,091.00","2,061.00",NaN,"97,099.00",5.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,967.00",1.00,NaN,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,NaN,"61,110,010,023,006.00",0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,14186244,NaN,NaN,NaN,2.00,3.00,None,NaN,2.00,NaN,NaN,"1,243.00","1,243.00",NaN,None,NaN,NaN,"6,059.00",NaN,2.00,2.00,440.00,NaN,NaN,"33,886,168.00","-117,823,170.00","8,432.00",1.00,NaN,NaN,NaN,1.00,122,261.00,None,"60,590,218.02","21,412.00","1,286.00",NaN,"97,078.00",6.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,962.00",1.00,NaN,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,NaN,"60,590,218,022,012.00",0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,NaN,NaN,NaN,3.00,4.00,None,8.00,3.00,NaN,NaN,"2,376.00","2,376.00",NaN,None,NaN,NaN,"6,037.00",NaN,3.00,NaN,NaN,NaN,2.00,"34,245,180.00","-118,240,722.00","13,038.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,LCR110000*,"60,373,001.00","396,551.00","3,101.00",NaN,"96,330.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,970.00",NaN,NaN,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,NaN,"60,373,001,001,006.00",-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,12095076,1.00,NaN,NaN,3.00,4.00,None,9.00,3.00,NaN,NaN,"2,962.00","2,962.00",NaN,None,NaN,NaN,"6,037.00",NaN,3.00,NaN,NaN,NaN,2.00,"34,145,202.00","-118,179,824.00","63,000.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,PSR2,"60,374,608.00","47,019.00","3,101.00","274,684.00","96,293.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,950.00",NaN,NaN,"276,684.00","773,303.00","2,016.00","496,619.00","9,516.26",None,NaN,"60,374,608,001,014.00",-0.00,2017-01-01,Central,None,None,Central,Single Family Residential,None,None


In [20]:
pd.DataFrame(df.isnull().sum())


,0
parcelid,0
airconditioningtypeid,39075
architecturalstyletypeid,54325
basementsqft,54348
bathroomcnt,0
...,...
buildingclassdesc,54395
heatingorsystemdesc,18611
propertylandusedesc,0
storydesc,54348


Nulls in columns

In [19]:
pd.DataFrame((df.isnull().sum()/df.shape[0]*100)) # Percentage of nulls in each column

,0
parcelid,0.00
airconditioningtypeid,71.84
architecturalstyletypeid,99.87
basementsqft,99.91
bathroomcnt,0.00
...,...
buildingclassdesc,100.00
heatingorsystemdesc,34.21
propertylandusedesc,0.00
storydesc,99.91


In [21]:
nulls_col = pd.DataFrame({'num_rows_missing': df.isnull().sum(), 
              'percent_rows_missing': (df.isnull().sum() / df.shape[0] * 100)})

nulls_col.sort_values(by='num_rows_missing', ascending=False)

,num_rows_missing,percent_rows_missing
buildingclasstypeid,54395,100.00
buildingclassdesc,54395,100.00
finishedsquarefeet15,54395,100.00
finishedsquarefeet13,54354,99.92
basementsqft,54348,99.91
...,...,...
latitude,0,0.00
fips,0,0.00
bedroomcnt,0,0.00
bathroomcnt,0,0.00


In [22]:
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing.sort_values(by='num_rows_missing', ascending=False)

Nulls by Row

In [23]:
df.isnull().sum(axis=1)

0        36
1        33
2        34
3        32
4        29
         ..
54390    34
54391    33
54392    32
54393    32
54394    34
Length: 54395, dtype: int64

In [24]:
df.head(1)[df.head(1).isna()]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None


In [25]:
df.isnull().sum(axis=1)


0        36
1        33
2        34
3        32
4        29
         ..
54390    34
54391    33
54392    32
54393    32
54394    34
Length: 54395, dtype: int64

In [26]:
df.isnull().sum(axis=1) / df.shape[1] * 100

0                      53.73
1                      49.25
2                      50.75
3                      47.76
4                      43.28
                ...         
54390                  50.75
54391                  49.25
54392                  47.76
54393                  47.76
54394                  50.75
Length: 54395, dtype: float64

In [27]:
nulls_row = pd.DataFrame({'num_cols_missing': df.isnull().sum(axis=1),
              'percent_cols_missing': df.isnull().sum(axis=1)/df.shape[1]*100})

nulls_row

,num_cols_missing,percent_cols_missing
0,36,53.73
1,33,49.25
2,34,50.75
3,32,47.76
4,29,43.28
...,...,...
54390,34,50.75
54391,33,49.25
54392,32,47.76
54393,32,47.76


In [28]:
nulls_row = df.merge(nulls_row,
                        left_index=True,
                        right_index=True)[['parcelid', 'num_cols_missing', 'percent_cols_missing']]

nulls_row

,parcelid,num_cols_missing,percent_cols_missing
0,14297519,36,53.73
1,17052889,33,49.25
2,14186244,34,50.75
3,12177905,32,47.76
4,12095076,29,43.28
...,...,...,...
54390,12412492,34,50.75
54391,11000655,33,49.25
54392,17239384,32,47.76
54393,12773139,32,47.76


In [29]:
nulls_row.sort_values(by='num_cols_missing', ascending=False)

,parcelid,num_cols_missing,percent_cols_missing
39062,14341728,48,71.64
9287,167686999,48,71.64
29088,167687839,48,71.64
11537,167689317,47,70.15
42974,17188862,47,70.15
...,...,...,...
41067,14292517,24,35.82
17445,14172935,24,35.82
38906,14193288,24,35.82
22863,14465914,23,34.33


In [30]:
def nulls_by_row(df):
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})
    rows_missing = df.merge(rows_missing,
                        left_index=True,
                        right_index=True)[['parcelid', 'num_cols_missing', 'percent_cols_missing']]
    return rows_missing.sort_values(by='num_cols_missing', ascending=False)

In [32]:
def summarize(df):
    '''
    summarize will take in a single argument (a pandas dataframe) 
    and output to console various statistics on said dataframe, including:
    # .head()
    # .info()
    # .describe()
    # .value_counts()
    # observation of nulls in the dataframe
    '''
    print('SUMMARY REPORT')
    print('=====================================================\n\n')
    print('Dataframe head: ')
    print(df.head(3))
    print('=====================================================\n\n')
    print('Dataframe info: ')
    print(df.info())
    print('=====================================================\n\n')
    print('Dataframe Description: ')
    print(df.describe())
    num_cols = [col for col in df.columns if df[col].dtype != 'O']
    cat_cols = [col for col in df.columns if col not in num_cols]
    print('=====================================================')
    print('DataFrame value counts: ')
    for col in df.columns:
        if col in cat_cols:
            print(df[col].value_counts(), '\n')
        else:
            print(df[col].value_counts(bins=10, sort=False), '\n')
    print('=====================================================')
    print('nulls in dataframe by column: ')
    print(nulls_by_col(df))
    print('=====================================================')
    print('nulls in dataframe by row: ')
    print(nulls_by_row(df))
    print('=====================================================')

In [33]:
summarize(df)

SUMMARY REPORT


Dataframe head: 
   parcelid  airconditioningtypeid  architecturalstyletypeid  basementsqft  \
0  14297519                    NaN                       NaN           NaN   
1  17052889                    NaN                       NaN           NaN   
2  14186244                    NaN                       NaN           NaN   

           bathroomcnt           bedroomcnt buildingclasstypeid  \
0                 3.50                 4.00                None   
1                 1.00                 2.00                None   
2                 2.00                 3.00                None   

   buildingqualitytypeid    calculatedbathnbr  decktypeid  \
0                    NaN                 3.50         NaN   
1                    NaN                 1.00         NaN   
2                    NaN                 2.00         NaN   

   finishedfloor1squarefeet  calculatedfinishedsquarefeet  \
0                       NaN                      3,100.00   
1                

Our accomplishments so far

We have:

- Acquired our data
- Examined the structure and integrity of the data
- We have observed univariate descriptive statistics
- We have examined null values (in total, by column, and by row)
- We have created functions for these processes


What we still need to do:

- Determine what columns can be saved by filling in nulls.
- Create functions that identify upper bound outliers and lower bound outliers
- Create functions that act on our outliers (drop them, compress them, etc.)
- Create functions/processes that act on nulls beyond simple removal


Fill in null values.

In [46]:
df["fireplacecnt"].fillna( 0 , inplace = True)

In [54]:
df['fireplaceflag'] = np.where(df.fireplacecnt > 0, 1 , 0)

In [55]:
df.fireplaceflag.value_counts()

0    47161
1     7234
Name: fireplaceflag, dtype: int64

In [48]:
df.fireplacecnt.value_counts()

0.00    47161
1.00     6025
2.00      942
3.00      229
4.00       35
5.00        3
Name: fireplacecnt, dtype: int64

In [60]:
df.loc[df['fireplacecnt'] > 0]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
1,17052889,NaN,NaN,NaN,1.00,2.00,None,NaN,1.00,NaN,"1,465.00","1,465.00","1,465.00",NaN,None,"1,465.00",NaN,"6,111.00",1.00,1.00,1.00,0.00,NaN,NaN,"34,449,266.00","-119,281,531.00","12,647.00",NaN,NaN,NaN,NaN,NaN,1110,261.00,None,"61,110,010.02","13,091.00","2,061.00",NaN,"97,099.00",5.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,967.00",1.00,1,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,NaN,"61,110,010,023,006.00",0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None
8,13944538,13.00,NaN,NaN,2.50,3.00,None,NaN,2.50,NaN,NaN,"1,340.00","1,340.00",NaN,None,NaN,NaN,"6,059.00",1.00,2.00,2.00,420.00,NaN,NaN,"33,771,438.00","-117,982,834.00","1,199.00",NaN,NaN,NaN,NaN,NaN,122,261.00,None,"60,590,888.01","24,832.00","1,286.00",NaN,"97,051.00",6.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,980.00",2.00,1,"96,617.00","319,668.00","2,016.00","223,051.00","4,078.08",None,NaN,"60,590,888,014,006.00",0.05,2017-01-02,Yes,None,None,None,Single Family Residential,None,None
12,14387959,NaN,NaN,NaN,2.50,3.00,None,NaN,2.50,NaN,NaN,"2,074.00","2,074.00",NaN,None,NaN,NaN,"6,059.00",1.00,2.00,2.00,485.00,NaN,NaN,"33,586,693.00","-117,697,202.00","5,600.00",NaN,NaN,NaN,NaN,NaN,122,261.00,None,"60,590,423.27","25,458.00","1,286.00",NaN,"96,971.00",7.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,978.00",2.00,1,"173,269.00","597,213.00","2,016.00","423,944.00","6,172.28",None,NaN,"60,590,423,273,002.00",-0.01,2017-01-02,None,None,None,None,Single Family Residential,None,None
16,14314879,NaN,NaN,NaN,4.50,6.00,None,NaN,4.50,NaN,NaN,"3,870.00","3,870.00",NaN,None,NaN,NaN,"6,059.00",2.00,4.00,3.00,720.00,NaN,NaN,"33,609,965.00","-117,850,693.00","10,050.00",1.00,NaN,NaN,NaN,1.00,122,261.00,None,"60,590,626.44","53,571.00","1,286.00","28,119.00","96,957.00",11.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,972.00",2.00,1,"298,263.00","2,040,000.00","2,016.00","1,741,737.00","21,758.26",None,NaN,"60,590,626,444,017.00",-0.00,2017-01-02,None,None,None,None,Single Family Residential,None,None
24,14344863,NaN,NaN,NaN,2.50,4.00,None,NaN,2.50,NaN,NaN,"1,979.00","1,979.00",NaN,None,NaN,NaN,"6,059.00",1.00,2.00,2.00,446.00,NaN,NaN,"33,713,632.00","-117,765,112.00","3,936.00",NaN,NaN,NaN,NaN,NaN,122,261.00,None,"60,590,525.25","52,650.00","1,286.00","276,486.00","96,954.00",8.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,977.00",2.00,1,"140,369.00","800,017.00","2,016.00","659,648.00","8,138.06",None,NaN,"60,590,525,253,000.00",-0.00,2017-01-02,None,None,None,None,Single Family Residential,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54025,14425750,NaN,NaN,NaN,2.00,4.00,None,NaN,2.00,NaN,NaN,"1,358.00","1,358.00",NaN,None,NaN,NaN,"6,059.00",1.00,2.00,2.00,428.00,NaN,NaN,"

In [37]:
# Data Dictionary of zillow data. Will help determine what to save
pd.set_option('display.max_colwidth', None)
raw_data_dictionary = pd.read_clipboard()
raw_data_dictionary

,Feature,Description
0,'airconditioningtypeid',Type of cooling system present in the home (if any)
1,'architecturalstyletypeid',"Architectural style of the home (i.e. ranch, colonial, split-level, etc…)"
2,'basementsqft',Finished living area below or partially below ground level
3,'bathroomcnt',Number of bathrooms in home including fractional bathrooms
4,'bedroomcnt',Number of bedrooms in home
5,'buildingqualitytypeid',Overall assessment of condition of the building from best (lowest) to worst (highest)
6,'buildingclasstypeid',"The building framing type (steel frame, wood frame, concrete/brick)"
7,'calculatedbathnbr',Number of bathrooms in home including fractional bathroom
8,'decktypeid',Type of deck (if any) present on parcel
9,'threequarterbathnbr',Number of 3/4 bathrooms in house (shower + sink + toilet)


Takeaways

- Fill in fireplaceflag with values 0 or 1. Filled in values does not match 'fireplacecnt' total.
- Fill in nulls with 0 in columns: 'basementsqft', 'fireplacecnt', 

- Drop 'hashottuborspa' due to column is the sum of columns 'pooltypeid2' and 'pooltypeid10'
- Drop 'poolcnt' due to sum (11447) does not match totalof columns of 'pooltypeid2' and 'pooltypeid7' (11374) short 73 pools, create new column 'haspool' with values 0 meaning No or 1 meaning yes.

In [61]:
df.poolcnt.value_counts()

1.00    11447
Name: poolcnt, dtype: int64

In [62]:
df.pooltypeid10.value_counts()

1.00    443
Name: pooltypeid10, dtype: int64

In [63]:
df.pooltypeid2.value_counts()

1.00    1071
Name: pooltypeid2, dtype: int64

In [64]:
df.pooltypeid7.value_counts()

1.00    10357
Name: pooltypeid7, dtype: int64

In [65]:
df.hashottuborspa.value_counts()

1.00    1514
Name: hashottuborspa, dtype: int64

In [66]:
df.loc[df['pooltypeid2'] > 0]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
46,14434171,NaN,NaN,NaN,6.50,5.00,None,NaN,6.50,NaN,NaN,"9,586.00","9,586.00",NaN,None,NaN,NaN,"6,059.00",0.00,6.00,4.00,"1,114.00",1.00,NaN,"33,496,171.00","-117,699,380.00","30,000.00",1.00,NaN,NaN,1.00,NaN,122,261.00,None,"60,590,423.32","25,459.00","1,286.00",NaN,"96,987.00",0.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,992.00",NaN,0,"1,708,622.00","3,624,442.00","2,016.00","1,915,820.00","38,491.98",None,NaN,"60,590,423,322,002.00",0.06,2017-01-02,None,None,None,None,Single Family Residential,None,None
147,14350624,NaN,NaN,NaN,4.00,6.00,None,NaN,4.00,NaN,NaN,"3,733.00","3,733.00",NaN,None,NaN,NaN,"6,059.00",0.00,4.00,2.00,464.00,1.00,NaN,"33,722,323.00","-117,744,913.00","7,448.00",1.00,NaN,NaN,1.00,NaN,122,261.00,None,"60,590,524.20","52,650.00","1,286.00","274,815.00","96,954.00",0.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,999.00",NaN,0,"427,940.00","799,634.00","2,016.00","371,694.00","10,794.22",None,NaN,"60,590,524,204,004.00",-0.02,2017-01-03,None,None,None,None,Single Family Residential,None,None
171,14394667,NaN,NaN,NaN,3.00,4.00,None,NaN,3.00,NaN,NaN,"3,309.00","3,309.00",NaN,None,NaN,NaN,"6,059.00",0.00,3.00,2.00,670.00,1.00,NaN,"33,567,399.00","-117,680,138.00","31,710.00",1.00,NaN,NaN,1.00,NaN,122,261.00,None,"60,590,423.33","25,458.00","1,286.00",NaN,"96,971.00",0.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,989.00",NaN,0,"567,646.00","1,452,674.00","2,016.00","885,028.00","14,914.64",None,NaN,"60,590,423,332,009.00",0.02,2017-01-03,None,None,None,None,Single Family Residential,None,None
189,14391477,NaN,NaN,NaN,2.50,4.00,None,NaN,2.50,NaN,NaN,"2,357.00","2,357.00",NaN,None,NaN,NaN,"6,059.00",0.00,2.00,2.00,587.00,1.00,NaN,"33,585,703.00","-117,744,295.00","4,420.00",1.00,NaN,NaN,1.00,NaN,122,261.00,None,"60,590,626.33","37,086.00","1,286.00",NaN,"96,974.00",0.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,995.00",NaN,0,"287,015.00","748,968.00","2,016.00","461,953.00","7,526.84",None,NaN,"60,590,626,332,010.00",0.01,2017-01-03,None,None,None,None,Single Family Residential,None,None
291,14081668,NaN,NaN,NaN,4.00,4.00,None,NaN,4.00,NaN,NaN,"3,186.00","3,186.00",NaN,None,NaN,NaN,"6,059.00",2.00,4.00,2.00,609.00,1.00,NaN,"33,724,009.00","-118,065,886.00","6,000.00",1.00,NaN,NaN,1.00,NaN,122,261.00,None,"60,590,995.15","25,218.00","1,286.00",NaN,"96,967.00",8.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,977.00",2.00,1,"343,287.00","651,462.00","2,016.00","308,175.00","7,274.26",None,NaN,"60,590,995,145,004.00",-0.01,2017-01-04,None,None,None,None,Single Family Residential,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53671,14461441,NaN,NaN,NaN,6.50,6.00,None,NaN,6.50,NaN,NaN,"4,767.00","4,767.00",NaN,None,NaN,NaN,"6,059.00",0.00,6.00,3.00,871.00,1.00,NaN

In [67]:
df.loc[df['pooltypeid7'] > 0]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
2,14186244,NaN,NaN,NaN,2.00,3.00,None,NaN,2.00,NaN,NaN,"1,243.00","1,243.00",NaN,None,NaN,NaN,"6,059.00",0.00,2.00,2.00,440.00,NaN,NaN,"33,886,168.00","-117,823,170.00","8,432.00",1.00,NaN,NaN,NaN,1.00,122,261.00,None,"60,590,218.02","21,412.00","1,286.00",NaN,"97,078.00",6.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,962.00",1.00,0,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,NaN,"60,590,218,022,012.00",0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,NaN,NaN,NaN,3.00,4.00,None,8.00,3.00,NaN,NaN,"2,376.00","2,376.00",NaN,None,NaN,NaN,"6,037.00",0.00,3.00,NaN,NaN,NaN,2.00,"34,245,180.00","-118,240,722.00","13,038.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,LCR110000*,"60,373,001.00","396,551.00","3,101.00",NaN,"96,330.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,970.00",NaN,0,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,NaN,"60,373,001,001,006.00",-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,12095076,1.00,NaN,NaN,3.00,4.00,None,9.00,3.00,NaN,NaN,"2,962.00","2,962.00",NaN,None,NaN,NaN,"6,037.00",0.00,3.00,NaN,NaN,NaN,2.00,"34,145,202.00","-118,179,824.00","63,000.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,PSR2,"60,374,608.00","47,019.00","3,101.00","274,684.00","96,293.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,950.00",NaN,0,"276,684.00","773,303.00","2,016.00","496,619.00","9,516.26",None,NaN,"60,374,608,001,014.00",-0.00,2017-01-01,Central,None,None,Central,Single Family Residential,None,None
7,11104527,1.00,NaN,NaN,3.00,4.00,None,8.00,3.00,NaN,NaN,"2,540.00","2,540.00",NaN,None,NaN,NaN,"6,037.00",0.00,3.00,NaN,NaN,NaN,2.00,"34,434,971.00","-118,464,390.00","10,384.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,SCUR2,"60,379,200.34","54,311.00","3,101.00","37,739.00","96,374.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,999.00",NaN,0,"271,715.00","430,108.00","2,016.00","158,393.00","6,103.36",None,NaN,"60,379,200,341,014.00",0.01,2017-01-02,Central,None,None,Central,Single Family Residential,None,None
16,14314879,NaN,NaN,NaN,4.50,6.00,None,NaN,4.50,NaN,NaN,"3,870.00","3,870.00",NaN,None,NaN,NaN,"6,059.00",2.00,4.00,3.00,720.00,NaN,NaN,"33,609,965.00","-117,850,693.00","10,050.00",1.00,NaN,NaN,NaN,1.00,122,261.00,None,"60,590,626.44","53,571.00","1,286.00","28,119.00","96,957.00",11.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,972.00",2.00,1,"298,263.00","2,040,000.00","2,016.00","1,741,737.00","21,758.26",None,NaN,"60,590,626,444,017.00",-0.00,2017-01-02,None,None,None,None,Single Family Residential,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54369,10867042,NaN,NaN,NaN,2.00,3.00,None,7.00,2.00,NaN,NaN,"2,141.00","2,141.00",NaN,None,NaN,NaN,"6,037.00",0.00,2.00

In [70]:
df[(df.pooltypeid7 > 0) | (df.pooltypeid2 > 0)]

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
2,14186244,NaN,NaN,NaN,2.00,3.00,None,NaN,2.00,NaN,NaN,"1,243.00","1,243.00",NaN,None,NaN,NaN,"6,059.00",0.00,2.00,2.00,440.00,NaN,NaN,"33,886,168.00","-117,823,170.00","8,432.00",1.00,NaN,NaN,NaN,1.00,122,261.00,None,"60,590,218.02","21,412.00","1,286.00",NaN,"97,078.00",6.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,962.00",1.00,0,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,NaN,"60,590,218,022,012.00",0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,NaN,NaN,NaN,3.00,4.00,None,8.00,3.00,NaN,NaN,"2,376.00","2,376.00",NaN,None,NaN,NaN,"6,037.00",0.00,3.00,NaN,NaN,NaN,2.00,"34,245,180.00","-118,240,722.00","13,038.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,LCR110000*,"60,373,001.00","396,551.00","3,101.00",NaN,"96,330.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,970.00",NaN,0,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,NaN,"60,373,001,001,006.00",-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,12095076,1.00,NaN,NaN,3.00,4.00,None,9.00,3.00,NaN,NaN,"2,962.00","2,962.00",NaN,None,NaN,NaN,"6,037.00",0.00,3.00,NaN,NaN,NaN,2.00,"34,145,202.00","-118,179,824.00","63,000.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,PSR2,"60,374,608.00","47,019.00","3,101.00","274,684.00","96,293.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,950.00",NaN,0,"276,684.00","773,303.00","2,016.00","496,619.00","9,516.26",None,NaN,"60,374,608,001,014.00",-0.00,2017-01-01,Central,None,None,Central,Single Family Residential,None,None
7,11104527,1.00,NaN,NaN,3.00,4.00,None,8.00,3.00,NaN,NaN,"2,540.00","2,540.00",NaN,None,NaN,NaN,"6,037.00",0.00,3.00,NaN,NaN,NaN,2.00,"34,434,971.00","-118,464,390.00","10,384.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,SCUR2,"60,379,200.34","54,311.00","3,101.00","37,739.00","96,374.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,999.00",NaN,0,"271,715.00","430,108.00","2,016.00","158,393.00","6,103.36",None,NaN,"60,379,200,341,014.00",0.01,2017-01-02,Central,None,None,Central,Single Family Residential,None,None
16,14314879,NaN,NaN,NaN,4.50,6.00,None,NaN,4.50,NaN,NaN,"3,870.00","3,870.00",NaN,None,NaN,NaN,"6,059.00",2.00,4.00,3.00,720.00,NaN,NaN,"33,609,965.00","-117,850,693.00","10,050.00",1.00,NaN,NaN,NaN,1.00,122,261.00,None,"60,590,626.44","53,571.00","1,286.00","28,119.00","96,957.00",11.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,972.00",2.00,1,"298,263.00","2,040,000.00","2,016.00","1,741,737.00","21,758.26",None,NaN,"60,590,626,444,017.00",-0.00,2017-01-02,None,None,None,None,Single Family Residential,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54369,10867042,NaN,NaN,NaN,2.00,3.00,None,7.00,2.00,NaN,NaN,"2,141.00","2,141.00",NaN,None,NaN,NaN,"6,037.00",0.00,2.00

In [75]:
def conditions(df):
    if (df['pooltypeid7'] > 0) or (df['pooltypeid2'] > 0) :
        return 1
    else:
        return 0

In [77]:
df['haspool'] = df.apply(conditions, axis=1)

In [78]:
df.haspool.value_counts()

0    42967
1    11428
Name: haspool, dtype: int64

In [81]:
df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,haspool
0,14297519,NaN,NaN,NaN,3.50,4.00,None,NaN,3.50,NaN,NaN,"3,100.00","3,100.00",NaN,None,NaN,NaN,"6,059.00",0.00,3.00,2.00,633.00,NaN,NaN,"33,634,931.00","-117,869,207.00","4,506.00",NaN,NaN,NaN,NaN,NaN,122,261.00,None,"60,590,630.07","53,571.00","1,286.00",NaN,"96,978.00",0.00,NaN,1.00,NaN,NaN,NaN,NaN,"1,998.00",NaN,0,"485,713.00","1,023,282.00","2,016.00","537,569.00","11,013.72",None,NaN,"60,590,630,072,012.00",0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None,0
1,17052889,NaN,NaN,NaN,1.00,2.00,None,NaN,1.00,NaN,"1,465.00","1,465.00","1,465.00",NaN,None,"1,465.00",NaN,"6,111.00",1.00,1.00,1.00,0.00,NaN,NaN,"34,449,266.00","-119,281,531.00","12,647.00",NaN,NaN,NaN,NaN,NaN,1110,261.00,None,"61,110,010.02","13,091.00","2,061.00",NaN,"97,099.00",5.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,967.00",1.00,1,"88,000.00","464,000.00","2,016.00","376,000.00","5,672.48",None,NaN,"61,110,010,023,006.00",0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None,0
2,14186244,NaN,NaN,NaN,2.00,3.00,None,NaN,2.00,NaN,NaN,"1,243.00","1,243.00",NaN,None,NaN,NaN,"6,059.00",0.00,2.00,2.00,440.00,NaN,NaN,"33,886,168.00","-117,823,170.00","8,432.00",1.00,NaN,NaN,NaN,1.00,122,261.00,None,"60,590,218.02","21,412.00","1,286.00",NaN,"97,078.00",6.00,NaN,NaN,NaN,NaN,NaN,NaN,"1,962.00",1.00,0,"85,289.00","564,778.00","2,016.00","479,489.00","6,488.30",None,NaN,"60,590,218,022,012.00",0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None,1
3,12177905,NaN,NaN,NaN,3.00,4.00,None,8.00,3.00,NaN,NaN,"2,376.00","2,376.00",NaN,None,NaN,NaN,"6,037.00",0.00,3.00,NaN,NaN,NaN,2.00,"34,245,180.00","-118,240,722.00","13,038.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,LCR110000*,"60,373,001.00","396,551.00","3,101.00",NaN,"96,330.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,970.00",NaN,0,"108,918.00","145,143.00","2,016.00","36,225.00","1,777.51",None,NaN,"60,373,001,001,006.00",-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None,1
4,12095076,1.00,NaN,NaN,3.00,4.00,None,9.00,3.00,NaN,NaN,"2,962.00","2,962.00",NaN,None,NaN,NaN,"6,037.00",0.00,3.00,NaN,NaN,NaN,2.00,"34,145,202.00","-118,179,824.00","63,000.00",1.00,NaN,NaN,NaN,1.00,0101,261.00,PSR2,"60,374,608.00","47,019.00","3,101.00","274,684.00","96,293.00",0.00,NaN,NaN,NaN,1.00,NaN,NaN,"1,950.00",NaN,0,"276,684.00","773,303.00","2,016.00","496,619.00","9,516.26",None,NaN,"60,374,608,001,014.00",-0.00,2017-01-01,Central,None,None,Central,Single Family Residential,None,None,1


In [85]:
df.pooltypeid10.value_counts()

1.00    443
Name: pooltypeid10, dtype: int64

In [ ]:
df["hashottuborspa"].fillna( 0 , inplace = True)
df["pooltypeid10"].fillna( 0 , inplace = True)
df["pooltypeid7"].fillna( 0 , inplace = True)
df["pooltypeid10"].fillna( 0 , inplace = True)
df["pooltypeid2"].fillna( 0 , inplace = True)
df["pooltypeid10"].fillna( 0 , inplace = True)

